---
## Rechnernutzung in der Physik  

**Institut für Experimentelle Teilchenphysik**  
Prof. G. Quast 
Dr. Th. Chwalek  
WS 2024/25 – Blatt 03  
Besprechung: Montag 2.12/ Dienstag 3.12. bzw. Montag 9.12./ Dienstag 10.12.

---

---
# Info-Veranstaltung zur Projektentwicklung Physik am 29.11.
---

Die Bereichsleitung BL5 organisiert am 29.11. um 16.00 Uhr im Gaede-HS eine große Informationsveranstaltung zur Projektentwicklung Physik. Dabei sollen von der Abteilung Planen und Bau die aktuellen Planungen zur baulichen Weiterentwicklung unserer Fakultät vorgestellt werden. Im Anschluss daran besteht die Möglichkeit, dies mit den drei anwesenden KIT-Präsidiumsmitgliedern Präsident Hesthaven, Vizepräsident Kraft und Vizepräsident Schwartze zu diskutieren. 
---



Auf dem dritten Übungsblatt gibt es zunächst eine Anwendung der Monte-Carlo-Methode zur Untersuchung der Poisson-Veteilung.
Anschließend beschäftigen Sie sich mit einem der wichtigsten Grundbegriffe der modernen Parameteranpassung - der Maximum-Likelihood-Methode.  
Sie haben mit großer Wahrscheinlichkeit im Laufe Ihres Studiums oder Privatlebens bereits die Maximum-Likelihood-Methode (oder ihre Abwandlungen) verwendet, um die Steigung einer Geraden oder andere Parameter einer Messung zu bestimmen. Mit diesem Übungsblatt beschäftigen Sie sich noch mal ganz genau mit der Methodik und nehmen hoffentlich mit, auf welchem Prinzip die unterschiedlichen eingebauten Fitfunktionen in *SciPy*, *kafe2*, *probfit* und vielen mehr (ja, auch Excel) aufgebaut sind. Sie werden merken, dass sich hinter all den angesprochenen Methoden keine schwarze Magie befindet, sondern die Umsetzung und Überführung auf eigene Programme ganz einfach ist. Mit einem guten Verständis der Maximum-Likelihood-Methode können Sie dann nicht nur die Methode der kleinsten Quadrate verstehen, sondern auch schon bald Anpassungen wie diese selbst durchführen.

---
# Aufgabe 1: Simulation eines Poisson-Prozesses <a id="Aufgabe1"></a>
---

In dieser Aufgabe soll eine Folge von Zeitpunkten erzeugt werden, zu denen ein Ereignis stattfindet; 
die Eintreffwahrscheinlichkeit pro Zeitintervall ist dabei konstant, d.h. der Prozess findet mit einer 
festen mittleren Rate statt. Solche Prozesse sind in der Physik sehr häufig, z.B. das Eintreffen eines
quantenmechanischen Ereignisses wie die Abstrahlung eines Photons durch ein angeregtes Atom oder 
Zerfälle von Kernen oder Teilchen oder die Zeitpunkte der Erzeugung von Teilchen in einem 
Teilchenbeschleuniger wie dem Large-Hadron-Collider. 

Für die Simulation von Experimenten zur Untersuchung von Poisson-Prozessen ist der Zeitpunkt des 
Ereignisses entscheidend, weil praktisch alle Sensoren zum Nachweis nach einem Ereignis für eine 
gewisse Zeit vollständig oder teilweise ineffizient sind oder ungenaue Ergebnisse liefern. 
Wenn z.B. ein elektrischer Puls einer bestimmten Dauer ausgelöst wird, dann kann kein zweiter 
Puls direkt anschließend fehlerfrei detektiert werden, weil es zu einer Überlagerung kommt. 

In dieser Studie soll zunächst eine Reihe von Zeitpnkten simuliert werden, zu denen Ereignisse 
stattgefunden haben.  Die Voraussetzung einer konstanten Eintreffwahrscheinlichkeit führt  
zu einer Poisson-Verteilung der in festen Zeitintervallen eintreffenden Ereignisse, wie 
im Dokument [Poisson.pdf](Poisson.pdf) illustriert wird.  

Nach einer Überprüfung der erwarteten statistischen Eigenschaften der erzeugten Zeitpunkte
wenden wir sie zur Bestimmung der Nachweiseffizienz und Korrektur für einen einfachen
Detektor an. 


## a) Verteilung der Wartezeiten

Zur Lösung der gestellen Aufgabe gehen wir zunächst von der Verteilung der sogenannten "Wartezeit" 
aus, d.h. der zwischen dem Auftreten von zwei Zufallsereignissen verstrichenen Zeit.

Die Verteilungsdichte der Wartezeiten, $t_w$, zwischen zwei
Ereignissen, die in der Zeit gleichverteilt sind, ist gegeben durch
$f(t_w)=\frac{1}{\overline{t_w}} \exp{(-t_w/\overline{t_w})}\,$, wobei
$\overline{t_w}$ die mittlere Wartezeit ist und dem Kehrwert der
Ereignisrate $R$ entspricht. Also gilt auch $f(t_w)=R \cdot \exp{(- R\,t_w)}$

  > **Herleitung: Verteilung der Wartezeiten**  
    Man denke sich die Zeit $t_w$ in $n$ kleine, gleich große Intervalle
    $\Delta t_i=t_w/n$ zerlegt. Die Wahrscheinlichkeit, ein Ereignis in
    einem solchen Intervall zu beobachten, ist gegeben durch
    $p=R\,t_w\,/n$. Ein Zerfall nach der Zeit $t_w$ bedeutet, dass in den
    Intervallen davor kein Ereignis beobachtet wurde, d.h.
    $p(t_w) = (1 - \, Rt_w/n)^n$, für $n \to\infty$ also $p(t_w) =\exp(-Rt_w)$.


Nutzen Sie diese Verteilung der Wartezeiten und schreiben Sie eine
Funktion, die, ausgehend von $t_0=0$, eine Folge von $N$=50'000
Zeitpunkten $t_i$ bestimmt, zu denen solche Ereignisse auftreten. Der
Einfachheit halber sei die Rate $R$\,=\,1/Zeiteinheit (ZE). 

Histogrammieren Sie zur Kontrolle die Zeiten zwischen
aufeinanderfolgenden Ereignissen und überlagern Sie zur Kontrolle 
die Exponentialfunktion.

Als kleine Erleichterung hier einige Funktionen aus den Vorlesungsbeispielen:

In [ ]:
# imports and helper functions
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sp


## Poisson-Verteilung aus Vorlesungsbeispielen
# ---------------------------------------
# Poisson distribution
def fPoisson(x, mu):
    k = np.around(x)
    #  return (mu**k)/np.exp(mu)/sp.gamma(k+1.)
    return np.exp(k * np.log(mu) - mu - sp.gammaln(k + 1.0))  # numerically more robust


## Funktion aus PhyPraKit
# ---------------------------------------
def histstat(binc, bine, pr=True):
    # calculate mean and standard deviation of a histogram 
    #  with bincontents binc and bin edges bine
    bincent = (bine[1:] + bine[:-1]) / 2  # determine bincenters
    mean = sum(binc * bincent) / sum(binc)
    rms = np.sqrt(sum(binc * bincent**2) / sum(binc) - mean**2)
    if pr:
        print('hist statistics:\n' '   mean=%g, sigma=%g\n' % (mean, rms))
    return mean, rms

In [ ]:
# --> Code zur Erzeugung der Zeitpunkte

Nevents = 50000
data = np.zeros(Nevents)
dT = 1.0  # mean time between signal events

# generate event times (signal)

# ->>

# plot time between events

# ->> 


## b) Statistische Auswertung

(1) Histogrammieren Sie die Häufigkeit von Ereignissen in festen Zeitintervallen, d.h. 
stellen Sie die zeitliche Verteilung der Ereigniszeipunkte in einem Histogramm mit
500 Bins dar. 
 
Welche Verteilung erhalten Sie? Welche Ereignisrate/Bin lesen Sie ab? Zeichen Sie 
auch die erwartete mittlere Rate ein. 

(2) Sellen Sie nun die Anzahlen der in den 500 Intervallen beobachteten Ereignisse,
d.h. die Bin-Inhalte aus (1), als Häufikeitsverteilung dar und überlagern Sie zum 
Vergleich die Poission-Verteilung.


In [ ]:
# statistical analysis

# (1) plot event rate in 500 bins
tbins = 500

# also show expectation value on graph

# (2) plot distribution of event counts per bin

# 
print('\n *==* Results PoissonProcess:')
print('Statistik der Ereignishäufigkeiten')
print(f'   mean of counts per bin: ???')
print(f'       standard deviation: ???')


## c) Anwendung: Totzeitkorrektur für einen Detektor

Als Anwendung unserer kleinen Experimentsimulation soll nun das Ansprechverhalten eines
Detektors und dessen Einfluss auf die gemessene Ereignisrate beschrieben werden.

Nehmen wir dazu an, ein Detektor habe nach dem Registrieren eines Ereignisses zum Zeitpunkt $t_i$ 
eine Totzeit, die dazu führt, dass die Nachweiseffizienz nach der Funktion 
 $\epsilon(t) = 5./ZE · (t − t_i )$ für $(t − ti ) < 0.2 ZE$ 
von Null bis Eins ansteigt. Welche Ereignisrate misst der Detektor? Wie groß
ist der auf Grund der Totzeit notwendige Korrekturfaktor auf die gemessene Rate?

 >  **Hinweis** 
    Der Code zur Detektorsimulation benötigt ein "Gedächtnis", nämlich den Zeitpunkt
    des letzten registrierten Ereignisses. Dazu empfiehlt sich die Implementierung
    als Klasse, wie im folgenden Codeabschnitt gezeigt: 

    ```python
    # Class defining detector properties
    class SimpleDetector:
    """a class defining a simple detector with dead time"""

    def __init__(self, tau=0.2):
    self.tlast = -1.
    # initialize variable storing argument of last call
    self.tfulleff = tau # time after which full efficiency is regained

    def Efficiency(self, dt):
    # function defining detector characteristics,
    #
    # ->  here the efficiency depending on time since last hit
    # ...
    return eff

    def Signal(self,t):
    # function returning True if signal at time t is detected
    #
    random sampling of function "Efficiency()"
    # ...
    #SigSeen = ... # True/False
    return SigSeen
    ```
    Eine Instanz der Klasse kann dann alle notwendigen Daten zum simulierten Detektor und zur
    Berechnung der Nachweiseffizienz speichern und die benötigten Funktionen bereit stellen.


In [ ]:
# Implementeriung der Klasse zur Beschreibung des Detektors 

# Class defining detector properties
class SimpleDetector:
    """a class defining a simple detector with dead time"""

# ->>


In [ ]:
# Totzeitbestimmung

# ->>

# print result:
print( 'Detector efficiency study:')
print( '  - events detected %i of %i , ' %(Nseen, Ndat ) )
print( '  - event rate %g, measured rate %g' %(Ndat/T, Nseen/T) )
print( '      ==> efficiency correction factor: %g' %(EffCorr) )


---
# Aufgabe 2: Poisson-Likelihood <a id="Aufgabe2"></a>
---

In der ersten Aufgabe beschäftigen Sie sich mit einem vermutlich bereits bekanntem Beispiel aus dem Praktikum: Sie haben eine radioaktive Quelle erhalten und messen elektronisch die Klicks eines Geiger-Müller-Zählrohrs in aufeinanderfolgenden, gleich langen Zeitintervallen. Die Messdaten haben Sie in der Datei *Geiger_Zaehler.txt* abgespeichert und wollen nun für die weitere Auswertung die Anzahl der radioaktiven Zerfälle pro Zeitintervall bestimmen. 

Bei der Auswertung der Daten erinnern Sie sich an die Vorlesung *Rechnernutzung* und die Parameterschätzung mithilfe der Maximum-Likelihood-Methode und beschließen, statt *kafe2* zu nutzen, die gesuchte Zahl selbst abzuschätzen. Sie schauen zurück in die Folien der Vorlesung und entdecken die Funktion

$$ 
L\left(\left\{x_i\right\}|a\right)=\prod_{i\leq n}f\left(x_i,a\right).
$$

Sie erinnern sich daran, dass $\left\{x_i\right\}$ ihre Messungen sind, $f(x,a)$ die zugrunde liegende Wahrscheinlichkeitsdichte und $a$ der gesuchte Parameter ist.

## a) Likelihoodfunktion

Im ersten Aufgabenteil überprüfen Sie graphisch die Form der Likelihoodfunktion.
> Frage: Warum ist es gerechtfertig die Poissonverteilung als zugrunde liegende Wahrscheinlichkeitsdichte für den Ausgang des Zählexperiments zu wählen? Welche Abweichungen von der Annahme gibt es beim Aufbau und der Messung?

->> Antwort:






Implementieren Sie die Likelihoodfunktion $L\left(\left\{x_i\right\}|a\right)$. Es bietet sich an, die Wahrscheinlichkeitsdichte als Argument der Funktion zu übergeben. Denken Sie daran, dass Sie einen gewissen Parameterereich abdecken müssen. Es bietet sich also an nicht nur $x$, sondern auch $a$ als Array zu behandeln.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sp

In [ ]:
# Poisson distribution
def fPoisson(x, nu_P):
    k=np.around(x)
    return (nu_P**k) / np.exp(nu_P) / sp.gamma(k+1.)

# Likelihood function
def Likelihood(x, a, func):
# ->>    

Betrachten Sie nun die Likelihoodfunktion für eine steigende Zahl an Messergebnissen $L\left(\left\{x_i\right\}|a\right)$. Stellen Sie dafür die Likelihoodfunktion für $a$ im Intervall $[2,9]$ dar für
* nur den ersten Messwert,
* den ersten und den zweiten Messwert,
* die ersten 10 Messwerte,
* die ersten 100 Messwerte.

Sie können mithilfe von *np.loadtxt("filename")* die Messdaten aus dem Verzeichnis laden, indem Sie das Notebook ausführen. Sollten Sie also das Notebook von ihrem eigenen PC in der Jupyter-Umgebung des SCC-Jupyter-Servers laden oder lokal auf Ihrem Rechner arbeiten, denken Sie daran, zuerst die Datei *Geiger_Zaehler.txt* herunterzuladen.

In [ ]:
#->>  your code comes here 
# load observations


# create parameter intervall

# list for desired observation numbers


# make plots


> Frage: Erläutern Sie das Verhalten des Maximums der Likelihoodfunktion. Schätzen Sie den Wert des gesuchten Paramters ab.

->> Antwort:






## b) Negative Log-Likelihood

In der Praxis verwendet anstelle von $L$ man die **Negative Log Likelihood (NLL)**

Beachten Sie die $y$-Skala bzw. den Wertebereich der Likelihoodfunktion für die vier verschiedenen Anzahlen an Messpunkten. Für eine große Anzahl an Messpunkten nimmt die Funktion immer kleinere Werte an. Eine Logarithmierung des Ausdruck vereinfacht nicht nur die  mathematische Analyse, sondern hilft auch numerisch, da das Produkt einer großen Anzahl kleiner Wahrscheinlichkeiten die numerische Genauigkeit des Computers für Fließkommaoperationen leicht unterschreiten kann, und dies wird gelöst, indem stattdessen die effizientere Summe der natürlichen Logarithmen der Wahrscheinlichkeiten ("Log Likelihood") berechnet wird. Statt nach dem Maximum der Likelihoodfunktion zu suchen, wird häufig nach dem Minimum der negativen Log Likelihood (NLL) gesucht. 

$$ 
-\ln\left(L\left(\left\{x_i\right\}|a\right)\right)=-\sum_{i\leq n}\ln\left(f\left(x_i,a\right)\right).
$$

Wiederholen Sie den ersten Aufgabenteil mit der NLL-Methode. Implementieren Sie dazu eine neue Funktion, die die NLL ausgibt. Betrachten Sie denselben Parameterbereich, aber diesmal für 
* nur den ersten Messwert,
* den ersten und den zweiten Messwert,
* die ersten 10 Messwerte,
* **alle Messwerte** statt nur 100 Messwerte.

Stellen Sie die $-\ln\left(L\left(\left\{x_i\right\}|a\right)\right)$ jeweils grafisch dar.

In [ ]:
# ->> your code, please !

# legative Log Likelihood function
def NLL(x, params, func, *args, **kwargs):
#->>

# make plots


## c) Parameterschätzung

Bestimmen Sie nun den gesuchten Parameter und schätzen Sie dessen $1\sigma$-Konfidenzintervall ab, indem Sie die *asymmetrischen* Grenzen für die Unsicherheit angeben.

Für den ersten Schritt kann die *NumPy*-Methode *np.argmin()* hilfreich sein. Sie gibt den Index des Arrayeintrags am Minimum aus. <br>
Den zweiten Schritt erhalten Sie, indem Sie diejenigen Punkte finden, deren NLL-Wert um $1/2$ größer sind als der des Maximums. Das $1\sigma$-Konfidenzintervall entspricht dann dem Wertebereich $[\hat{a}-\Delta^-,\hat{a}+\Delta^+]$. Die üblich Angabe für den besten Schätzwert erfolgt dann über folgende Darstellung:
$$\hat{a}^{+\Delta^+}_{-\Delta^-}.$$
Hier eine schnelle Illustration:

In [ ]:
x = np.linspace(0, 10, 1000)
y = (x - 3)**2 + 0.2

dots_x = [3 - 1/np.sqrt(2), 3, 3 + 1/np.sqrt(2)]
dots_y = [0.7, 0.2, 0.7]

plt.plot(x, y, 'b-', label='NLL')
plt.hlines(0.7, 1, 5, color='green', label=r'NLL$[\hat{a}]$+0.5')
plt.plot(dots_x, dots_y, 'ro')

colors = ['gold', 'orange', 'tomato']
labels = [r'Unterschranke $\hat{a}-\Delta^-$', r'Minimum $\hat{a}$', r'Oberschranke $\hat{a}+\Delta^+$']
for i in range(3):
    plt.vlines(dots_x[i], 0, dots_y[i], linestyle='dashed', color=colors[i], label=labels[i])

plt.ylim(-0.1, 1)
plt.xlim(1, 5)
plt.xlabel(r'$a$')
plt.ylabel(r'NLL $-\ln L\left(\left\{x_i\right\},a\right)$')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

Es bietet sich an die Zwischenschritte als Funktion zu implementieren, damit sie erneut verwendet werden können.

In [ ]:
# ->> your own code
def find_estimate(nlLH, par):


a_best, a_left, a_right = find_estimate(NLL(obs, a, fPoisson), a)
print(f'Der gesuchte Parameter konnte geschätzt werden zu {a_best:.3f}+{a_right:.3f}-{a_left:.3f}')